<a href="https://colab.research.google.com/github/fregean/data_analytics_with_python/blob/section2-1/cleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 データクレンジング

##  データの入手・結合・表示

In [292]:
# pip install mojimoji

In [293]:
import requests
from bs4 import BeautifulSoup
import os
import time

import pandas as pd
import numpy as np

import datetime
import mojimoji
import re

from typing import Union

pd.set_option('display.max_columns', 100)

In [294]:
URL = 'https://www.meti.go.jp/policy/economy/distribution/daikibo/todokede.html'
res = requests.get(URL)
soup = BeautifulSoup(res.content, 'lxml') # バイト文字列を読み込んで文字コードを推定してくれる # 0.03秒

In [295]:
DIR = './drive/MyDrive/data/'
 
if not os.path.exists(DIR):
    # ディレクトリが存在しない場合、ディレクトリを作成する
    os.makedirs(DIR)

In [296]:
for t in soup.select('a'):

  if '法第5条第1項（EXCEL形式' in t.text:
    href = t.get('href')

    if 'daikibo' in href:
      link = 'https://www.meti.go.jp' + href
    else:
      link = 'https://www.meti.go.jp/policy/economy/distribution/daikibo/' + href

    file = requests.get(link)
    filename = os.path.basename(link)

    with open(DIR + filename, 'wb') as f:
      f.write(file.content)

      time.sleep(3)

In [297]:
import glob
 
files = glob.glob(DIR + 'ritti_todogai*.xlsx')
files_lst = []

for file in files:

    input_book = pd.ExcelFile(file) 
    input_sheet_name = input_book.sheet_names
    data = input_book.parse(input_sheet_name[0], header=1,
                 skiprows=[0,1,2,3,4,5,6,7], usecols='A:D, G, J:M',
                 names=['局名', '都道府県', '大規模小売店舗名', '所在地', 
                        '小売業者名', '届出日', '開店予定日', '店舗面積', '取下げ'])
    files_lst.append(data)
 
total_data = pd.concat(files_lst)
input_file_name = DIR + 'total.xlsx'
total_data.to_excel(input_file_name, index=False)

In [298]:
DIR='drive/MyDrive/data/'
input_file_name = DIR + 'total.xlsx'
df = pd.read_excel(input_file_name, sheet_name=0)
# df # 12667 rows × 9 columns

In [299]:
# 大まかにデータフレームの属性を掴む

# print(df.shape)
# print('---------')
# print(df.info())

## 不要な列の削除・欠損値の処理

In [300]:
# 「取下げ」列で欠損値でない行のインデックスを抽出
drop_lst = df[df['取下げ'].notna()].index
# 取下げられた届出のインデックスなので、その行は分析に不要（異常値）のため、削除する
df.drop(index=drop_lst, inplace=True) 

# df # 12392 rows × 9 columns

In [301]:
# 「取下げ」列は欠損値のみ。不要になったので、この列を削除する
df.drop(columns='取下げ', inplace=True)

# df # 12392 rows × 8 columns

In [302]:
# それ以外の列で欠損値を含む行を確認する

# df[df.isnull().any(axis=1)] # 70 rows × 8 columns

In [303]:
# 推測して埋められる行もあるが、今回は欠損値を含む行は削除する
df.dropna(how='any', inplace=True)

# df # 12322 rows × 8 columns

In [304]:
# 欠損値が残っていないかチェックする
# df.isnull().sum()

In [305]:
# インデックスが所々抜けているのでふり直す

# df = df.reset_index(drop=True)

In [306]:
# 新しいインデックス確認

# df # 12322 rows × 8 columns

## データ型の変更

pandasは日付をデフォルトで読み込むと文字列型になる

日付データ型へ変更する

In [307]:
# データ型を確認する
# 「届出日」「開店予定日」がdatetime型でない
# https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-types

# df.dtypes

In [308]:
# object型の「届出日」データをそのままdatetime型へ変換しようとするとエラーになる

#pd.to_datetime(df['届出日'], format='%Y-%m-%d')

### 届出日

上のセルを実行してエラーが出る原因は誤った日付の形式が混ざっていることらしい<br>
他に、シリアル値という値も混ざっている（csvシートで確認する）<br>
まずは「届出日」列をシリアル値とそうでないものに分ける

In [309]:
# シリアル値が含まれるインデックス（フラグ）と、数をカウントする関数
from typing import Union

def check_serial(col: pd.Series) -> Union[pd.Series, int]:

    flag = col.astype('str').str.isdigit()

    return flag, flag.sum()

In [310]:
flg, flg_num = check_serial(df['届出日'])
# print(f'フラグ：\n{flg}')
# print('-----------------')
# print(f'フラグの数：{flg_num}')

In [311]:
# Length: 170
# print(df['届出日'][flg]) # ちなみにシリアル値一つ一つはint型
# print('-----------------')
# Length: 12152
# print(df['届出日'][~flg]) # ~は否定を表すので、flgが立っていないインデックスを表す

In [312]:
# int型のシリアル値を、datetime型の%Y-%m-%d形式に書き換える

def replace_serial_to_date(col: pd.Series, flag: bool) ->  pd.Series:

    series = pd.to_timedelta(col[flag].astype('float') - 2, unit='D')+pd.to_datetime('1900/1/1')

    return series

In [313]:
# string型の値を、datetime型の%Y-%m-%d形式に書き換える
# errors='coerce'オプションで、エラーが出る値はNaTに置き換える

def replace_str_to_date(col: pd.Series, flag: bool) ->  pd.Series:

    series = pd.to_datetime(col[flag], format='%Y-%m-%d', errors='coerce')

    return series

In [314]:
from_serial = replace_serial_to_date(df['届出日'], flg)
from_str = replace_str_to_date(df['届出日'], ~flg)

# Length: 170
# print(from_serial) # ちなみにシリアル値一つ一つはint型
# print('-----------------')
# # Length: 12152
# print(from_str) 

In [315]:
# 処理後に欠損値はできてないか確認
# print(from_serial[from_serial.isnull()].index)

# 処理中のエラーで欠損値になった値が6つあった
# print(from_str[from_str.isnull()].index)

In [316]:
# 元のデータを確かめると、異常値だった行が欠損値になった様子

# df['届出日'][from_str[from_str.isnull()].index]

In [317]:
# 異常値の処理を行う関数

import logging

def processing_outlier(word: str) -> datetime:

    season_dic = {'上旬': '5日', '中旬': '15日', '下旬': '25日'}

    for key, value in season_dic.items():
        word = word.replace(key, value)

    word = re.sub(r',| ', '', word)
    #数字以外ハイフンへ置換
    word = re.sub(r'\D', '-', word)
    #末尾のハイフン削除
    word = word.rstrip('-')
    # データ型変換
    try:
        word = datetime.datetime.strptime(word, '%Y-%m-%d')
    except:
        word = np.nan
        logging.exception('適切な形式ではありません')

    return word


In [318]:
series = df['届出日'][from_str[from_str.isnull()].index]
converted_words = [processing_outlier(w) for w in series]

# converted_words

In [319]:
# 異常値を正常値に変換

from_str_part = pd.Series(data=converted_words, index=from_str[from_str.isnull()].index, name='届出日')
# from_str_part

In [320]:
# 2つのSeriesを結合する（from_strの欠損値は、from_str_partの同じインデックスの値で上書き）
from_str = from_str.combine_first(other=from_str_part)

# from_str # 12152

In [321]:
# 上書きされていることを確認する
# from_str.loc[from_str_part.index]

In [322]:
# from_serialとfrom_strを結合する（両者はインデックスが異なるので、上書きは起きない）
# 結合したSeriesで元のDataFrasmeの「届出日」列を上書きする

df['届出日'] = from_serial.combine_first(other=from_str)
# df['届出日']

### 開店予定日
届出日と同じ処理を行う

In [323]:
# 「開店予定日」列にシリアル値がいくつあるか数える

flg, flg_num = check_serial(df['開店予定日'])
# print(f'フラグ：\n{flg}')
# print('-----------------')
# print(f'フラグの数：{flg_num}')

In [324]:
# Length: 182
# print(df['開店予定日'][flg]) # ちなみにシリアル値一つ一つはint型
# print('-----------------')
# Length: 12140
# print(df['開店予定日'][~flg]) # ~は否定を表すので、flgが立っていないインデックスを表す

In [325]:
# int型のシリアル値をdatetime型へ変換
# from_serial = replace_serial_to_date(df['開店予定日'], flg)
# str型をdatetime型へ変換
# from_str = replace_str_to_date(df['開店予定日'], ~flg)

# Length: 182
# print(from_serial) # ちなみにシリアル値一つ一つはint型
# print('-----------------')
# Length: 12140
# print(from_str) 

In [326]:
# 処理後に欠損値はできてないか確認
# print(from_serial[from_serial.isnull()].index)

# 処理中のエラーで欠損値になった値が30個あった
# print(from_str[from_str.isnull()].index)

In [327]:
# 元のデータを確かめると、異常値だった行が欠損値になった様子

# df['開店予定日'][from_str[from_str.isnull()].index]

In [328]:
series = df['開店予定日'][from_str[from_str.isnull()].index]
converted_words = [processing_outlier(w) for w in series]

# converted_words

In [329]:
# 関数processing_outlierで処理できないイレギュラーな異常値は欠損値にした
#（欠損値としないで、これらも処理できる関数を作るか、あるい直接的に直しても良い）

from_str_part = pd.Series(data=converted_words, index=from_str[from_str.isnull()].index, name='開店予定日')
# from_str_part

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


In [330]:
# 2つのSeriesを結合する（from_strの欠損値をfrom_str_partの同じインデックスの値で上書き）
from_str = from_str.combine_first(other=from_str_part)

# from_str

In [331]:
# 上書きされていることを確認する
# from_str.loc[from_str_part.index]

In [332]:
# from_serialとfrom_strを結合する（両者はインデックスが異なるので上書きは起きない）
# 結合したSeriesで元のDataFrasmeの「開店予定日」列を上書きする

df['開店予定日'] = from_serial.combine_first(other=from_str)
# df['開店予定日']

In [333]:
# 欠損値を削除する
df.dropna(how='any', inplace=True)
df.reset_index(drop=True, inplace=True)

In [334]:
# df # 12307 rows × 8 columns

## 適切な表記への置換

### 所在地の表記を整形する

In [335]:
import logging

def clean_address(col: pd.Series) -> list:

    places = []
    for idx, place in enumerate(col):
        
        try:

            if '外' in place:
                place = place[:place.rfind('外')]
            elif 'ほか' in place:
                place = place[:place.rfind('ほか')]
            elif '他' in place:
                place = place[:place.rfind('他')]   

            place = ''.join(place.split())

        except:

            place = np.nan
            logging.exception('適切な住所ではありません')
            print(f'インデックス：{idx}')

        places.append(place)
      
    return places

In [336]:
# 住所の不要な表記を削除する関数実行（返り値はなし）
# エラーになったらそこはNaNで埋める
places = clean_address(df['所在地'])

ERROR:root:適切な住所ではありません
Traceback (most recent call last):
  File "<ipython-input-335-a4cf8af997ac>", line 10, in clean_address
    if '外' in place:
TypeError: argument of type 'int' is not iterable


インデックス：8459


In [337]:
# リストをインデックス付きのSeriesに変換する
places = pd.Series(data=places, index=df.index, name='所在地')
# NaNのあるインデックス8459を確認
# print(places[places.isnull()])

In [338]:
# Seriesを上書きする
df['所在地'] = places

In [339]:
# df[df['所在地'].isnull()]

In [340]:
# この行を削除する
df.dropna(how='any', inplace=True)

In [341]:
# インデックス振り直し
df.reset_index(drop=True, inplace=True)

In [342]:
# df # 12306 rows × 8 columns

## 表記の統一


### 小売業者名の表記の統一

In [343]:
# len(df['小売業者名'].unique())

In [344]:
# 頻出する不要な表記を削除する関数

def clean_labels(data: str, replace_at: str = '') -> str:
        
    data = re.sub('株式会社|㈱|（株）|\(|(株)|\)|ほか|他', replace_at, data)

    data = ''.join(data.split())

    return data

In [345]:
# 横棒、中黒記号の類似を統一する関数

def replace_symbol(data: str, replace_line: str = 'ー', replace_point: str = '・') -> str:
    
    data = re.sub('－|-|‐|−|‒|—|–|―|ｰ|─|━|ㅡ|ـ|⁻|₋', replace_line, data)
    data = re.sub('･|・|・|・|•', replace_point, data)

    return data

In [346]:
# 表記ゆれ、不要な表記の修正

def fix_spelling(col: pd.Series) -> list:

    retailer = []
    for d in col:
        
        d = clean_labels(d, replace_at='')
        d = replace_symbol(d, replace_line='ー', replace_point='・')
        d = d.replace('“', '') # ダブルコロン削除
        d = d.upper() # 大文字に統一
        d = mojimoji.han_to_zen(d) # 半角に統一
                
        if 'セブン' in d:
            d = re.sub(r'セブンイレブン', 'セブンーイレブン', d)
            d = re.sub('・ジャパン|ジャパン', '', d)
            
        if 'ルシア' in d:
            d = re.sub(r'ウェルシア', 'ウエルシア', d)
            d = d.replace('薬局', '')
            
        if 'アイ・ティー' in d:
            d = re.sub(r'アイ・ティーエックス', 'アイ・ティー・エックス', d)
            
        # 業者名が連名になっているところは改行する
        if '、' in d:
            d = d.replace('、', '・\n')       

        retailer.append(d)
        
    return retailer

In [347]:
retailer = fix_spelling(df['小売業者名'])

In [348]:
# リストをインデックス付きのSeriesに変換する
retailer = pd.Series(data=retailer, index=df.index, name='小売業者名')
# このSeriesで「小売業者名」を上書きする
df['小売業者名'] = retailer

In [349]:
# pd.DataFrame(df['小売業者名'].value_counts())

### 都道府県名の表記の統一

市区町村名を都道府県名に置換

In [350]:
# 都道府県名に市の名前が混ざっている

# print(df['都道府県'][df['都道府県'].str.contains('市')])
# print('----------------')
# print(df['都道府県'][df['都道府県'].str.contains('市')].unique())
# print(len(df['都道府県'][df['都道府県'].str.contains('市')].unique()))

In [351]:
# 「市名」になった値を特定するフラグを作る
city_flg = df['都道府県'].str.contains('市')
# city_flg

In [352]:
# 上記を除くと、さらに「県」表記が抜けているものがある
#  '富山', '石川', '沖縄'の3パターン

# df['都道府県'][~city_flg].unique()

In [353]:
# ダウンロードurlからzipファイルを取得する
"""
住所データcsv
http://jusyo.jp/csv/new.php
"""

URL = 'http://jusyo.jp/downloads/new/csv/csv_zenkoku.zip'

res = requests.get(URL)
filename = os.path.basename(URL)

with open(DIR + filename, 'wb') as f:
    f.write(res.content)

In [354]:
# zipファイル解凍
import zipfile

files = glob.glob(DIR + filename)

with zipfile.ZipFile(files[0]) as existing_zip:
    existing_zip.extractall(DIR)

In [355]:
zenkoku = pd.read_csv(DIR + 'zenkoku.csv', encoding='cp932')
# zenkoku

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [356]:
# 市区町村名から都道府県名を推定する関数
# 関数のtype hint未記入（returnがnumpyデータ型になるパターンをまだ調べていないため）
def city_to_prefectures(data, city):     

    rows = data[data['市区町村'].str.contains(city)]

    city = rows['都道府県'].unique()

    return city

In [357]:
# 関数の挙動確認

# city_to_prefectures(zenkoku, '北九州市')

In [358]:
# 都市と都道府県の変換辞書
cities = df['都道府県'][df['都道府県'].str.contains('市')].unique()

# 広島市は北海道と広島県の候補がある
city_dic = {city : city_to_prefectures(zenkoku, city) for city in cities}
# city_dic

In [359]:
# 元のDataFrameで広島市を確認する
# 中国地方の広島市のみ
# df['局名'][df['都道府県']=='広島市'].unique()

In [360]:
# 上書きする
city_dic['広島市'] = np.array(['広島県'], dtype=object)
# city_dic

In [361]:
# 辞書のvalueについていたarrayを外す
city_dic = {k: v[0] for k, v in city_dic.items()}
# city_dic

In [362]:
# 置換辞書を用いて一括で置換する
df['都道府県'].replace(city_dic, inplace=True)
# df['都道府県'][city_flg]

In [363]:
# 47都道府県のリストを作成
local = [ v for v in df['都道府県'][~city_flg].unique() if len(v)>2 ]
# print(local)
# print(len(local))

In [364]:
# print(df['都道府県'][~df['都道府県'].isin(local)])
temp = df['都道府県'][~df['都道府県'].isin(local)].unique()
# temp

In [365]:
# 「県」表記が抜けている都道府県名を補完するための置換辞書を作成する

temp_dic = {t: t+'県' for t in temp}
# temp_dic

In [366]:
# 置換辞書で上書きする
df['都道府県'].replace(temp_dic, inplace=True)

### 局名の表記の統一
局名に含まれる異常値を修正

In [367]:
# df['局名'].unique()

In [368]:
# fix_spellingでなくとも空白除去でこと足りる

block = [''.join(d.split()) for d in df['局名']]

In [369]:
# df.isnull().sum()

In [370]:
# リストをインデックス付きのSeriesに変換する
block = pd.Series(data=block, index=df.index, name='局名')
# このSeriesで「小売業者名」を上書きする
df['局名'] = block
# df['局名'].unique()

In [371]:
# 九地方区分名以外の表記が混ざっている
# print(df['局名'].unique())
# print('-----------------')
# print(df['局名'].unique()[-2:])

In [372]:
# 誤った局名の表記をerr_変数に格納する
err_1 = df['局名'].unique()[-1] 
err_2 = df['局名'].unique()[-2] 

# 局名をerr_変数で指定した行を確認する
# df[(df['局名']==err_1) | (df['局名']==err_2)]

In [373]:
# 誤った局名の表記に対応する都道府県名を抽出する
# print(df['都道府県'][df['局名']==err_1])
# print(df['都道府県'][df['局名']==err_2])

# 誤った局名の表記に対応する都道府県名をlocal_変数に格納する
local_1 = df['都道府県'][df['局名']==err_1]
local_2 = df['都道府県'][df['局名']==err_2]

In [374]:
# これらの都道府県名に対応する一番多い（正しい）局名を抽出する
# print(df['局名'][df['都道府県']== local_1.values[0]].value_counts().index[0])
# print(df['局名'][df['都道府県']== local_2.values[0]].value_counts().index[0])

# これらの都道府県名に対応する一番多い（正しい）局名をblock_変数に格納する
block_1 = df['局名'][df['都道府県']== local_1.values[0]].value_counts().index[0]
block_2 = df['局名'][df['都道府県']== local_2.values[0]].value_counts().index[0]

In [375]:
# 誤った局名を正しい局名に変換する置換辞書を作成する
block_dic = {err_1 : block_1, err_2 : block_2}

# 置換辞書で置き換えを実行
df['局名'].replace(block_dic, inplace=True)

In [376]:
# 局名が正しい九つの地方区分名のみになっている
# df['局名'].unique()

In [377]:
# データ型の確認
# df.dtypes

## 欠損値の確認
csv保存し、読み込み直すことでスペース記号になっている値を欠損値として発見できる

In [378]:
# 欠損値なし
# df.isnull().sum()

In [379]:
DIR = 'drive/MyDrive/data/'

# 欠損値処理まで済んだデータをsection_2.csvとして保存しておく
# インデックスは出力しない
df.to_csv(DIR + 'section_2.csv', index=False)

In [380]:
df = pd.read_csv(DIR + 'section_2.csv', parse_dates=['届出日', '開店予定日'])
# df # 12321 rows × 9 columns

In [381]:
# df.dtypes

In [382]:
# 読み込み直すと、なぜか欠損値が一つ見つかった
# 元々、入力がスペース記号のものが混じっていたため
# df.isnull().sum()

In [383]:
# df[df['小売業者名'].isnull()]

In [384]:
# 欠損値を含む行を削除
df.dropna(how='any', inplace=True)
# df # 12320 rows × 9 columns

In [385]:
# 欠損値処理まで済んだデータをsection_2.csvとして保存し直し
# インデックスは出力しない
df.to_csv(DIR + 'section_2.csv', index=False)

In [386]:
df = pd.read_csv(DIR + 'section_2.csv', parse_dates=['届出日', '開店予定日'])
# df # 12320 rows × 9 columns

In [387]:
# 今度は欠損値ゼロになっている
# df.isnull().sum()

## データクレンジングはここで終了